In [1]:
# valuation_date時におけるcodeで指定した銘柄の個別銘柄分析
# SBI証券のモバイルの銘柄分析を参考に作成

In [2]:
### パラメータ設定セル
from datetime import date
from dateutil.relativedelta import relativedelta

# パラメータをセット
# code: 証券コード
# valuation_date: 過去にさかのぼって評価日をdatetime.dateでセット可能。デフォルト値は今日(date.today())
code = 4478
valuation_date = date.today()
# valuation_date = date(2024, 7, 30)


In [3]:
# 読み込みファイルパスの設定とimportしたいmoduleパス(pythonパス)の設定
from pathlib import Path
import os, sys

CURRENT_DIR = Path(os.getcwd())
PJ_DIR = CURRENT_DIR.parent.parent
LIB_DIR = PJ_DIR / "lib"
DATA_DIR = PJ_DIR / "data" 

sys.path.append(str(LIB_DIR))

# notebook内で利用するmoduleのimport
from lib_dataprocess import PricelistPl, FinancequotePl, KessanPl, KessanFig, PricelistFig, read_data, get_df_latest_yearly_performance, get_latest_stockprice, print_finance_quote
from lib_dataprocess import get_fig_expected_performance_progress_rate_pycharts, MeigaralistPl, get_fig_actual_performance_progress_rate_pycharts, ShikihoPl
import polars as pl

# dataの読み込み
fp = DATA_DIR/"raw_pricelist.parquet"
df = read_data(fp)
PPL = PricelistPl(df)

# dataの読み込み
fp = DATA_DIR/"reviced_pricelist.parquet"
df = read_data(fp)
RevPPL = PricelistPl(df)

fp = DATA_DIR/"finance_quote.parquet"
df = read_data(fp)
FPL = FinancequotePl(df)
FPL.with_columns_market_cap()

fp = DATA_DIR/"kessan.parquet"
df = read_data(fp)
KPL = KessanPl(df)

fp = DATA_DIR/"meigaralist.parquet"
df = read_data(fp)
MPL = MeigaralistPl(df)

fp = DATA_DIR/"shikiho.parquet"
df = read_data(fp)
SPL = ShikihoPl(df)

In [4]:
### 最新情報
# 指定したcodeの指定した日における株価と各種ファンダメンタルズデータをまとめて標準出力する
# pricelist_dfは、raw_pricelistかreviced_pricelistかケースに応じて使い分ける。
# def print_finance_quote(
#        pricelist_df: pl.DataFrame,
#        finance_quote_df: pl.DataFrame,
#        code: int, 
#        valuation_date: date=date.today()
#    ) -> None:
print_finance_quote(PPL.df, FPL.df, code, valuation_date=valuation_date)
print()
SPL.print_stock_df(code, num=4, valuation_date=valuation_date)

フリー(4478)の銘柄情報

終値: 3135.0円(2024年12月27日)
予想配当利回り: 0.0%(2024年12月27日)
予想PER: None倍(2024年12月27日)
実績PBR: 10.9倍(2024年12月27日)
自己資本比率: 42.0%(2024年12月27日)
予想ROE: 決算予想がないため、表示不可
予想ROA: 決算予想がないため、表示不可
時価総額: 1845.3億円(2024年12月27日)

4478(フリー)の四季報データ履歴

発行日: 2024年12月18日
【均衡圏】
  柱のERPは人事労務が人気で、中堅企業の追加契約が順調。中小向けの値上げも寄与。開発費の資産計上再開による押し上げ効果が想定超。営業損益均衡圏まで増額。買収時の一括減損特損消える。
【加速】
  会計事務所経由の新規加入が増加。今後も税務申告や使い勝手などの機能強化を継続。25年6月期の従業員は純増100～200人に抑制(前期は423人)。

発行日: 2024年09月13日
【黒字近づく】
  柱のERPは中堅企業の追加契約が順調に推移、一部製品の値上げも寄与。個人向け堅調増。増収効果と広告宣伝費圧縮、開発費の資産計上などで営業赤字幅が急減。買収時の一括減損特損消える。
【効率化】
  請求書や法人カードの従量課金型、顧客自身による追加購入など営業の効率化を模索。制度対応が一服、蓄積した見込み客への再提案に重点置いて生産性向上。

発行日: 2024年06月14日
【赤字激減】
  高単価な中堅企業向けが順調増。人員増や広告費増などの先行費用重い。25年6月期は中堅企業の追加契約が大幅増。費用増も徹底的に抑制し、営業赤字幅は急縮小。買収関連の減損特損消える。
【転換点】
  25年6月期に調整後利益で上場後初の黒字化が視野。開発費の資産計上開始、固定資産や買収時のれんの毎四半期減損処理も終了。7月に会計製品を価格改定。

発行日: 2024年03月16日
【赤字続く】
  制度改正を背景に高単価の中堅企業向けが順調増。ただ人員増や広告費増などの先行費用重い。なお営業赤字。買収関連特損。25年6月期は追加契約が大幅増。費用増負担も一服。赤字幅急縮小。
【借り入れ】
  25年6月期に調整後利益で黒字化視野。32億円を借り入れ、カード事業強化。決済

In [5]:
### 業績推移

# KessanFig(
#    code: int, 
#    settlement_type: Literal["通期", "四半期"], 
#    output_target: str = "jupyter",
#    start_settlement_date: date = date(1900, 1, 1),
#    end_settlement_date: date = date(2999, 12, 31)
#)

KFIG = KessanFig(code, "通期", end_settlement_date=valuation_date)
KFIG.add_trace_profits()
KFIG.fig.show()

In [6]:
# 今季決算予想発表の推移
df = KPL.get_settlement_forcast(code, valuation_date)
df

code,settlement_date,settlement_type,announcement_date,sales,operating_income,ordinary_profit,final_profit,reviced_eps,dividend,quater
i64,date,str,date,i64,i64,i64,i64,f64,f64,i64


In [7]:
### 通期業績、業績予想
# def get_df_latest_yearly_performance(code: int, valuation_date: date=date.today()) -> pl.DataFrame:
df = get_df_latest_yearly_performance(code, valuation_date)
df

フリー(4478)の通期決算(評価日：2024年12月28日)


決算期,決算発表日,売上高,営業利益,経常利益,純利益,EPS,1株配当
str,date,i64,i64,i64,i64,f64,f64
"""2024年6月期""",2024-08-14,25430,-8386,-8638,-10150,-174.4,0.0


In [8]:
KFIG = KessanFig(code, "四半期", end_settlement_date=valuation_date)
KFIG.add_trace_profits()
KFIG.fig.show()
KPL.get_latest_quater_settlement(code, valuation_date)

code,settlement_date,settlement_type,announcement_date,sales,operating_income,ordinary_profit,final_profit,reviced_eps,dividend,quater
i64,date,str,date,i64,i64,i64,i64,f64,f64,i64
4478,2024-09-30,"""四""",2024-11-14,7376,213,163,169,2.9,2.9,1
4478,2024-06-30,"""四""",2024-08-14,7007,-1426,-1465,-1514,-26.0,-20.4,4
4478,2024-03-31,"""四""",2024-05-15,6652,-2245,-2289,-2430,-41.8,-33.7,3
4478,2023-12-31,"""四""",2024-02-14,6051,-2641,-2758,-3918,-67.5,-43.6,2
4478,2023-09-30,"""四""",2023-11-14,5720,-2074,-2126,-2288,-39.5,-36.3,1
4478,2023-06-30,"""四""",2023-08-14,5406,-3216,-3225,-4495,-78.6,-59.5,4
4478,2023-03-31,"""四""",2023-05-15,5095,-2080,-2099,-2957,-51.8,-40.8,3
4478,2022-12-31,"""四""",2023-02-14,4480,-1452,-1461,-1620,-28.5,-32.4,2


In [9]:
# codeで指定した銘柄のevaluation_dateで指定した時点での最新の年度決算予想に基づく
# 売上高~純利益の決算進捗率を円グラフで表示するためのfigを返す
# from plotly.graph_objects import Figure
# def get_fig_expected_performance_progress_rate_pycharts(code: int, evaluation_date: date=date.today()) -> Figure:

fig1 = get_fig_expected_performance_progress_rate_pycharts(code, valuation_date)
fig1.show()

2024-12-28におけるフリー(4478)の2025年6月期の決算予想が公表されていないため、決算進捗率を表示できません。
フリー(4478)の2025年6月期第1四半期決算進捗率(評価日：2024-12-28)


In [10]:
last_year_valuation_date = date(valuation_date.year-1, valuation_date.month, valuation_date.day)

# codeで指定した銘柄のevaluation_dateで指定した時点での最新の四半期決算発表に基づく
# 売上高~純利益の決算進捗率を円グラフで表示するためのfigを返す
# valuation_dateで指定する日において、当年度の本決算が発表されていない日を指定した場合は前期末第4四半期の進捗率を表示するので、進捗率100%で表示される。
fig2 = get_fig_actual_performance_progress_rate_pycharts(code, last_year_valuation_date, KPL.df, MPL.df)
fig2.show()

フリー(4478)の2024年6月期第1四半期決算進捗率(評価日：2023-12-28)


In [11]:
#### valuation_dateに過去日を指定した場合、valuation_dateを含む期の株価推移
#### 答え合わせ

In [12]:
# 株価推移
# 前期算発表翌営業日～今期決算発表日翌営業日
start_date, end_date = KPL.get_current_settlement_period_by_announcement_date(code, valuation_date, "四")

df = RevPPL.df.filter(pl.col("code")==code).filter(pl.col("date")>start_date)
start_date = df["date"].to_list()[0]

df = RevPPL.df.filter(pl.col("code")==code).filter(pl.col("date")>end_date)
if not end_date == date(2999, 12, 31):
    end_date = df["date"].to_list()[0]

PFIG = PricelistFig(code, RevPPL.df, MPL.df, start_date, end_date)
df = RevPPL.df.filter(pl.col("code")==code).filter(pl.col("date")<=valuation_date)
vline_date = df["date"].to_list()[-1]
PFIG.add_vline(vline_date)
PFIG.fig.show()

In [13]:
# appendix
KPL.df.filter(pl.col("code")==code).filter(pl.col("announcement_date")>=date(2024,1,1))

code,settlement_date,settlement_type,announcement_date,sales,operating_income,ordinary_profit,final_profit,reviced_eps,dividend,quater
i64,date,str,date,i64,i64,i64,i64,f64,f64,i64
4478,2023-12-31,"""四""",2024-02-14,6051,-2641,-2758,-3918,-67.5,-43.6,2
4478,2024-03-31,"""四""",2024-05-15,6652,-2245,-2289,-2430,-41.8,-33.7,3
4478,2024-06-30,"""本""",2024-08-14,25430,-8386,-8638,-10150,-174.4,0.0,4
4478,2024-06-30,"""四""",2024-08-14,7007,-1426,-1465,-1514,-26.0,-20.4,4
4478,2024-09-30,"""四""",2024-11-14,7376,213,163,169,2.9,2.9,1


In [14]:
PFIG = PricelistFig(code, RevPPL.df, MPL.df, date(2023, 12, 1), date.today())
PFIG.fig.show()

In [15]:
# 次の決算どうだったか
KFIG = KessanFig(code, "四半期", end_settlement_date=valuation_date+relativedelta(days=100))
KFIG.add_trace_profits()
KFIG.fig.show()

In [16]:
from lib_dataprocess import CreditbalancePl

CPL = CreditbalancePl()
CPL.with_columns_margin_ratio()
CPL.df.filter(pl.col("code")==code)

code,date,unsold_margin,purchase_margin,margin_ratio
i64,date,i64,i64,f64
4478,2019-12-20,6200,717500,115.73
4478,2020-01-03,400,729300,1823.25
4478,2020-01-10,1500,600400,400.27
4478,2020-01-17,2100,450600,214.57
4478,2020-01-24,100,528300,5283.0
…,…,…,…,…
4478,2024-11-22,180100,465500,2.58
4478,2024-11-29,227500,386500,1.7
4478,2024-12-06,272200,284400,1.04
